In [1]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.supabase import SupabaseVectorStore
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI
from langchain.output_parsers import RegexParser

In [3]:
from utils.utils import initialize_environment_variables, initialize_subabase_client, initialize_openai_client

initialize_environment_variables("../.env")

# Initialize Supabase Client
supabase_client = initialize_subabase_client()

# Initialize OpenAI Client
openai_client = initialize_openai_client()

In [4]:
embeddings = OpenAIEmbeddings()

In [5]:
vector_store = SupabaseVectorStore(
        client=supabase_client,
        embedding=embeddings,
        table_name="vector_store"
    )

In [6]:
query = "Was ist Unterricht?"

In [7]:
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

This should be in the following format:

Question: [question here]
Helpful Answer: [answer here]
Score: [score between 0 and 100]

Begin!

Context:
---------
{context}
---------
Question: {question}
Helpful Answer:"""
output_parser = RegexParser(
    regex=r"(.*?)\nScore: (.*)",
    output_keys=["answer", "score"],
)
PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"],
    output_parser=output_parser
)

In [8]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="map_rerank", return_intermediate_steps=True, prompt=PROMPT)

In [9]:
query_embeddings = embeddings.embed_query(query)

2024-01-29 22:40:48,277:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [10]:
relevant_chunks = vector_store.similarity_search_by_vector_with_relevance_scores(query_embeddings,k=2)

2024-01-29 22:40:52,060:INFO - HTTP Request: POST https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/rpc/match_documents?limit=2 "HTTP/1.1 200 OK"


In [11]:
chunk_docs=[]
for chunk in relevant_chunks:
        chunk_docs.append(chunk[0])

In [12]:
results = chain({"input_documents": chunk_docs, "question": query})

/home/cts/.cache/pypoetry/virtualenvs/unterricht-python-backend-9Hag_kgR-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/cts/.cache/pypoetry/virtualenvs/unterricht-python-backend-9Hag_kgR-py3.10/lib/python3.10/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
2024-01-29 22:40:58,919:INFO - HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


In [13]:
text_reference=""
for i in range(len(results["input_documents"])):
    text_reference+=results["input_documents"][i].page_content
output={"Answer":results["output_text"],"Reference":text_reference}

In [14]:
output

{'Answer': ' Unterricht ist eine pädagogische Maßnahme, bei der Lehrkräfte den Schülerinnen und Schülern Wissen und Fähigkeiten vermitteln und sie beim Lernen unterstützen. Der Unterricht in der Grundschule ist geprägt vom Klassleiterprinzip, fächerverbindendem Unterricht und einer rhythmisierten Struktur, die den Bedürfnissen der Kinder angepasst ist. Regelmäßige Rückmeldungen und Gespräche über Gelerntes und Geleistetes sind Teil des Unterrichts und dienen der individuellen Förderung der Schülerinnen und Schüler. ',
 'Reference': '26Bildungs- und Erziehungsauftrag der Grundschule\nDie Schülerinnen und Schüler zeigen so persönliche Stärken, die für ihre Persönlichkeitsentwicklung \nund ihr Lernen bedeutsam sind. Die Lehrkräfte der Grundschule nehmen diese bewusst wahr, \nbeachten neben Ergebnissen auch die Leistungen, die in den Lern- und Arbeitsprozessen erbracht \nwerden, und unterstützen die Kinder beim Aufbau und der Erweiterung ihrer Kompetenzen. Sie \nermöglichen ihnen Erfolgser

In [17]:
def getanswer(query):
    embeddings = OpenAIEmbeddings()
    vector_store = SupabaseVectorStore(
        client=supabase_client,
        embedding=embeddings,
        table_name="vector_store"
    )
    query_embeddings = embeddings.embed_query(query)
    relevant_chunks = vector_store.similarity_search_by_vector_with_relevance_scores(query_embeddings,k=2)
    chunk_docs=[]
    for chunk in relevant_chunks:
        chunk_docs.append(chunk[0])
    results = chain({"input_documents": chunk_docs, "question": query})
    text_reference=""
    for i in range(len(results["input_documents"])):
        text_reference+=results["input_documents"][i].page_content
    output={"Answer":results["output_text"],"Reference":text_reference}
    return output

In [18]:
getanswer("Was ist Unterricht?")

2024-01-29 22:42:29,329:INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-01-29 22:42:29,485:INFO - HTTP Request: POST https://smxwnqdpxcvclxhbvhsf.supabase.co/rest/v1/rpc/match_documents?limit=2 "HTTP/1.1 200 OK"
/home/cts/.cache/pypoetry/virtualenvs/unterricht-python-backend-9Hag_kgR-py3.10/lib/python3.10/site-packages/langchain/chains/llm.py:344: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
2024-01-29 22:42:31,665:INFO - HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"


{'Answer': ' Unterricht ist eine pädagogische Maßnahme, bei der Lehrkräfte den Schülerinnen und Schülern Wissen und Fähigkeiten vermitteln und sie beim Lernen unterstützen. Der Unterricht in der Grundschule ist geprägt vom Klassleiterprinzip, fächerverbindendem Unterricht und einer rhythmisierten Struktur, die den Bedürfnissen der Kinder angepasst ist. Regelmäßige Rückmeldungen und Gespräche über Gelerntes und Geleistetes sind Teil des Unterrichts und dienen der individuellen Förderung der Schülerinnen und Schüler. ',
 'Reference': '26Bildungs- und Erziehungsauftrag der Grundschule\nDie Schülerinnen und Schüler zeigen so persönliche Stärken, die für ihre Persönlichkeitsentwicklung \nund ihr Lernen bedeutsam sind. Die Lehrkräfte der Grundschule nehmen diese bewusst wahr, \nbeachten neben Ergebnissen auch die Leistungen, die in den Lern- und Arbeitsprozessen erbracht \nwerden, und unterstützen die Kinder beim Aufbau und der Erweiterung ihrer Kompetenzen. Sie \nermöglichen ihnen Erfolgser